In [16]:
# app.py
import json
from flask import Flask, request, jsonify
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import tokenizer_from_json


app = Flask(__name__)

# 모델과 토크나이저, 레이블 인코더 로드
model = load_model('./data/my_model.keras')
# JSON 파일에서 Tokenizer 로드
with open('./data/tokenizer.json') as json_file:
    tokenizer_json = json.load(json_file)
tokenizer = tokenizer_from_json(tokenizer_json)

# JSON 파일에서 LabelEncoder 로드
with open('./data/label_encoder.json') as json_file:
    label_encoder_classes = json.load(json_file)
label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(label_encoder_classes)

@app.route('/recommend', methods=['POST'])
def recommend_jobs():
    data = request.json
    skills = data.get('skills')
    num_recommendations = data.get('num_recommendations', 3)
    
    sequences = tokenizer.texts_to_sequences([skills])
    max_len = model.input_shape[1]
    padded_sequence = pad_sequences(sequences, maxlen=max_len)
    padded_sequence = np.reshape(padded_sequence, (padded_sequence.shape[0], padded_sequence.shape[1], 1))
    
    predictions = model.predict(padded_sequence)
    top_indices = predictions[0].argsort()[-num_recommendations:][::-1]
    recommended_jobs = label_encoder.inverse_transform(top_indices)
    
    return jsonify({'recommended_jobs': recommended_jobs.tolist()})

if __name__ == '__main__':
    app.run('127.0.0.1',port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Jun/2024 14:37:07] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2024 14:37:31] "POST /case2 HTTP/1.1" 400 -
